# Decrypting a substitution cypher

High-level algorithm:
- We incrementally find the solution via a [backtracking](https://en.wikipedia.org/wiki/Backtracking) algorithm.
- The partial solutions ("candidates") are built up using a letter frequency heuristic
- Candidates are abandoned when they result in words that are (mostly) not in the given English dictionary

In [1]:
import collections
import nltk.corpus
import re

We use the default words corpus in the `/usr/share/dict/words` accessed [via NLTK](https://www.nltk.org/book/ch02.html), with some modification.

In [2]:
# convert to all uppercase set
ENGLISH_DICT = set(w.upper() for w in nltk.corpus.words.words())
# remove single letters except for 'A' and 'I'
# this creates a huge improvement since one-letter words in the cyphertext
# quickly remove many potential candidate branches
for ch in 'BCDEFGHJKLMNOPQRSTUVWXYZ':
    ENGLISH_DICT.remove(ch)

In [3]:
TEXT = re.sub(r'\s+', ' ', '''
    LRVMNIR BPR SUMVBWVR JX BPR LMIWV YJERYRKBI JX QMBM 
    WI BPR XJVNI MKD YMIBRUT JX IRHX WI BPR RIIRKVR JX 
    YMBINLMTMIPW UTN QMUMBR DJ W IPMHH BUT BJ RHNVWDMBR 
    BPR YJERYRKBI JX BPR QMBM MVVJUDWKO BJ YT WKBRUSURBMBWJK 
    LMIRD JK XJUBT TRMUI JX IBNDT WB WI KJB MK RMIT BMIQ 
    BJ RASHMWK RMVP YJERYRKB MKD WBI IWOKWXWVMKVR MKD 
    IJYR YNIB URYMWK NKRASHMWKRD BJ OWER M VJYSHRBR 
    RASHMKMBWJK JKR CJNHD PMER BJ LR FNMHWXWRD MKD WKISWURD 
    BJ INVP MK RABRKB BPMB PR VJNHD URMVP BPR IBMBR JX 
    RKHWOPBRKRD YWKD VMSMLHR JX URVJOKWGWKO IJNKDHRII IJNKD 
    MKD IPMSRHRII IPMSR W DJ KJB DRRY YTIRHX BPR XWKMH 
    MNBPJUWBT LNB YT RASRUWRKVR CWBP QMBM PMI HRXB KJ DJNLB
    BPMB BPR XJHHJCWKO WI BPR SUJSRU MSSHWVMBWJK MKD 
    WKBRUSURBMWJK W JXXRU YT BPRJUWRI WK BPR PJSR BPMB BPR 
    RIIRKVR JX JQWKMCMK QMUMBR CWHH URYMWK WKBMVB
'''.strip())

In [4]:
# helper function
def substitute(cyphertext, key):
    s = ''
    for ch in cyphertext:
        if ch.strip():
            s += key[ch] if key[ch] else '_'
        else:
            s += ' '
    return s

### The algorithm

In [8]:
def solve(cyphertext, english_dict, threshold=0.85):
    """
    Decrypts any English language-based cyphertext encrypted using substitution.
    
    Parameters
    ----------
        cyphertext : str
            the cypher to be decrypted
        threshold : float
            the percentage of words captured by a given candidate solution s that
            *must* be in the English dictionary in order for s to *not* be
            abandoned as a candidate solution.
    """
    
    # English letters sorted by frequency
    # see: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html
    LETTERS_SORTED_BY_FREQ = "EARIOTNSLCUDPMHGBFYWKVXZJQ"
    
    # initialize solution dict
    solution = dict.fromkeys(LETTERS_SORTED_BY_FREQ)

    cypher_letters_sorted_by_freq = [
        t[0] for t in sorted(
            collections.Counter(cyphertext).items(), 
            key=lambda t: t[1],
            reverse=True
        )
        if t[0] in LETTERS_SORTED_BY_FREQ
    ]
    
    def _assign(cypher_letters, remaining_letters):
        """
        Recursively built the key `solution` to the cyphertext
        by assigning `cypher_letters[0]` to a letter in `remaining_letters`.
        
        Returns True iff a satisfactory solution has been found.

        Note: 
            - `cypher_letters` is sorted according to letter frequency in the 
                cyphertext.
            - `remaining_letters` is sorted according to letter frequency in the
                English language at-large.
        """

        if not cypher_letters:
            # all letters assigned
            return True

        ch1 = cypher_letters.pop(0)

        # try all letters in `remaining_letters` in order
        # until a satisfactory solution has been found
        # (i.e. a solution that doesn't result in any deadends)
        i = -1
        while i < len(remaining_letters) - 1:
            i += 1

            solution[ch1] = remaining_letters[i]

            if _deadend():
                # unassign
                solution[ch1] = None
                continue

            ch2 = remaining_letters.pop(i)
            if _assign(cypher_letters, remaining_letters):
                return True

            # else: backtrack
            # put the current character back in list
            remaining_letters.insert(i, ch2)
            solution[ch1] = None

        # no satisfactory solution was found (in this branch)
        # reset before returning
        cypher_letters.insert(0, ch1)
        return False

    def _deadend():
        """
        Returns True if the current partial solution has reached a deadend.
        
        A deadend occurs iff *less than* `threshold` percent of words (that are
        completely spelled out by the candidate solution) occur in the given 
        English dictionary.
        """
        text2 = substitute(cyphertext, solution)
        # get all words that are completely spelled out via the key
        words = set([w for w in text2.split() if '_' not in w])
        
        if not words:
            return False

        # if more than `threshold` % are words in dictionary, *not* a deadend
        if sum(w in english_dict for w in list(words)) / len(words) > threshold:
            return False

        return True

    _assign(cypher_letters_sorted_by_freq, list(LETTERS_SORTED_BY_FREQ))
    
    return solution

### Solution

In [6]:
solution = solve(TEXT, ENGLISH_DICT)

In [7]:
substitute(TEXT, solution)

'BECAUSE THE PRACTICE OF THE BASIC MOVEMENTS OF KATA IS THE FOCUS AND MASTERY OF SELF IS THE ESSENCE OF MATSUBAYASHI RYU KARATE DO I SHALL TRY TO ELUCIDATE THE MOVEMENTS OF THE KATA ACCORDING TO MY INTERPRETATION BASED ON FORTY YEARS OF STUDY IT IS NOT AN EASY TASK TO EXPLAIN EACH MOVEMENT AND ITS SIGNIFICANCE AND SOME MUST REMAIN UNEXPLAINED TO GIVE A COMPLETE EXPLANATION ONE WOULD HAVE TO BE ZUALIFIED AND INSPIRED TO SUCH AN EXTENT THAT HE COULD REACH THE STATE OF ENLIGHTENED MIND CAPABLE OF RECOGNIJING SOUNDLESS SOUND AND SHAPELESS SHAPE I DO NOT DEEM MYSELF THE FINAL AUTHORITY BUT MY EXPERIENCE WITH KATA HAS LEFT NO DOUBT THAT THE FOLLOWING IS THE PROPER APPLICATION AND INTERPRETAION I OFFER MY THEORIES IN THE HOPE THAT THE ESSENCE OF OKINAWAN KARATE WILL REMAIN INTACT'